<a href="https://colab.research.google.com/github/brettmorrisonAO5ANNEX/Super-Resolution-Implementation/blob/main/SRCNN_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
import tensorflow_datasets as tfds

# Data Preprocessing

## Load Data

In [5]:
def preprocess(example):
    # LR = low-res image (input), HR = high-res image (label)
    lr = tf.image.convert_image_dtype(example['lr'], tf.float32)
    hr = tf.image.convert_image_dtype(example['hr'], tf.float32)

    # Upscale the LR image to match HR size (x4 for DIV2K)
    lr_upscaled = tf.image.resize(lr, size=tf.shape(hr)[:2], method='bicubic')

    return lr_upscaled, hr

# Load pre-defined splits directly
train_data = tfds.load('div2k/bicubic_x4', split='train', shuffle_files=True)
val_data = tfds.load('div2k/bicubic_x4', split='validation', shuffle_files=True)

# Apply preprocessing
train_data = train_data.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_data = val_data.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# Prepare batches
BATCH_SIZE = 1
train_data = train_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_data = val_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

EXTRACTING {'train_lr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X4.zip', 'valid_lr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X4.zip', 'train_hr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip', 'valid_hr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip'}


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/div2k/bicubic_x4/incomplete.W9EHZN_2.0.0/div2k-train.tfrecord*...:   0%|  …

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/div2k/bicubic_x4/incomplete.W9EHZN_2.0.0/div2k-validation.tfrecord*...:   …

Dataset div2k downloaded and prepared to /root/tensorflow_datasets/div2k/bicubic_x4/2.0.0. Subsequent calls will reuse this data.



# Creating The Model

In [6]:
def SRCNN():
    model = Sequential()
    # Patch extraction and representation
    model.add(Conv2D(64, (9, 9), activation='relu', padding='same', input_shape=(None, None, 3)))
    # Non-linear mapping
    model.add(Conv2D(32, (1, 1), activation='relu', padding='same'))
    # Reconstruction
    model.add(Conv2D(3, (5, 5), activation='linear', padding='same'))
    return model

srcnn = SRCNN()
srcnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Training & Evaluation

In [ ]:
srcnn.fit(train_data, validation_data=val_data, epochs=50)

Epoch 1/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1637s 2s/step - accuracy: 0.7253 - loss: 0.0116 - val_accuracy: 0.8268 - val_loss: 0.0035
Epoch 2/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 172s 214ms/step - accuracy: 0.8603 - loss: 0.0041 - val_accuracy: 0.9102 - val_loss: 0.0032
Epoch 3/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 173s 216ms/step - accuracy: 0.8763 - loss: 0.0034 - val_accuracy: 0.8530 - val_loss: 0.0035
Epoch 4/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 212s 229ms/step - accuracy: 0.8466 - loss: 0.0037 - val_accuracy: 0.8677 - val_loss: 0.0031
Epoch 5/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 192s 217ms/step - accuracy: 0.8730 - loss: 0.0034 - val_accuracy: 0.8761 - val_loss: 0.0034
Epoch 6/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 202s 216ms/step - accuracy: 0.8692 - loss: 0.0033 - val_accuracy: 0.8960 - val_loss: 0.0030
Epoch 7/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 203s 217ms/step - accuracy: 0.8839 - loss: 0.0030 - val_accuracy: 0.8569 - val_loss: 0.0035
Epoch 8/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 204s 219ms/step - accuracy: 0.8758 - l

# Test Output

In [ ]:
def load_and_preprocess_image(path, upscale_size=None):
    # Load image
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)

    # Resize (optional, if not already the upscaled size)
    if upscale_size:
        img = tf.image.resize(img, upscale_size, method='bicubic')

    # Convert to float32 and normalize
    img = tf.image.convert_image_dtype(img, tf.float32)

    # Add batch dimension
    img = tf.expand_dims(img, axis=0)  # shape: (1, h, w, 3)

    return img

# Load the low-res upscaled image
input_img = load_and_preprocess_image("willy_14.png")

# Predict high-res version using SRCNN
output = srcnn.predict(input_img)

# Remove batch dimension for visualization
output_img = tf.squeeze(output, axis=0)

# Show original (input) and super-resolved (output) side by side
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Input (Bicubic Upscaled)")
plt.imshow(tf.squeeze(input_img))
plt.axis('off')
plt.subplot(1, 2, 2)
plt.title("SRCNN Output")
plt.imshow(output_img)
plt.axis('off')
plt.show()